# 03 Transform V2 Fight Details to add Percentages and Differentials
- Calculate historical average of fight stats for each fight
- Everything is from the perspective of the F1 fighter
- Stats for the F1 fighter are basically an "offensive" rating for the F1 fighter
- Stats for the F2 fighter are basically an inverse "defensive" rating for the F1 fighter
- We also want to look at the difference in the stats for each fight, and then historical average difference

## Imports

In [2]:
import pandas as pd
import re

## Pull in V2_Fight_Details CSV

In [3]:
df = pd.read_csv('../../02_Data/02_Processed_Data/V2_Fight_Details.csv',index_col=0)

#### Notes on this initial set of data:
- No nulls currently since I dropped them all already
- No time in position details since they had nulls
- Only 1 side.  Not flipped and appended yet

## Need to add and flip before the transformations

In [4]:
# Setup Columns Again
F1_Columns = [col for col in df.columns if 'f1' in col.lower()]
F2_Columns = [col for col in df.columns if 'f2' in col.lower()]
Other_Columns = [col for col in df.columns if not 'f2' in col.lower() and not 'f1' in col.lower()]

Ordered_Columns = Other_Columns + F1_Columns + F2_Columns
Flipped_Columns = Other_Columns + F2_Columns + F1_Columns

# Put Columns in Order
df = df[Ordered_Columns]

# Create Flipped df
flipped_df = df[Flipped_Columns]
flipped_df.columns = Ordered_Columns

# Concatenate df and flipped_df
df = pd.concat([df, flipped_df])

# Reset the index
df = df.reset_index().drop(columns=['index'])

## Setup Column References for Easy Access  *Do not delete*

In [5]:
# These columns are used in the transformations
F1_Columns = [col for col in df.columns if 'f1' in col.lower()]
F2_Columns = [col for col in df.columns if 'f2' in col.lower()]
Other_Columns = [col for col in df.columns if not 'f2' in col.lower() and not 'f1' in col.lower()]

F1_Strikes = [col for col in F1_Columns if 'strikes' in col.lower()] + ['F1_Knock_Down_Landed']
F1_Grappling = [col for col in F1_Columns if 'grappling' in col.lower()]
F1_TIP = [col for col in F1_Columns if 'tip' in col.lower()]
F1_Identification = ['F1_FighterID','F1_Name'] # This is what its supposed to be incase I add more columns
F1_Identification = list(set(F1_Columns) - set(F1_Strikes) - set(F1_Grappling) - set(F1_TIP))

F2_Strikes = [col for col in F2_Columns if 'strikes' in col.lower()] + ['F2_Knock_Down_Landed']
F2_Grappling = [col for col in F2_Columns if 'grappling' in col.lower()]
F2_TIP = [col for col in F2_Columns if 'tip' in col.lower()]
F2_Identification = ['F2_FighterID','F2_Name']
F2_Identification = list(set(F2_Columns) - set(F2_Strikes) - set(F2_Grappling) - set(F2_TIP))

## Perform Transformation:  Calculate success percent for every pair of words that have "landed" and "attempted"

In [6]:
def Perform_calc_success_percent(df):
    Both_Landed_Attempts, Attempts_Only, Landed_Only = get_attempts_landed_columns(df)
    num_columns_before = df.shape[1]
    columns_before = df.columns
    print(f'num columns before: {num_columns_before}')
    for col in Both_Landed_Attempts:
        df = calc_success_percent(df, col)
    num_columns_after = df.shape[1]
    columns_after = df.columns
    print(f"# Columns Added: {num_columns_after-num_columns_before}")
    print(f'num columns after: {num_columns_after}')
    new_columns = list(set(columns_after) - set(columns_before))
    return df, new_columns

In [7]:
def calc_success_percent(df, col_root):
    attempts = col_root + '_attempts'
    landed = col_root + '_landed'
    success_percent = col_root + '_percent'
    
    df[success_percent] = df.apply(lambda x: x[landed]/x[attempts] if x[attempts] != 0 else 0, axis=1)
    
    return df

#calc_success_percent(df, 'F1_Body_Significant_Strikes')

In [8]:
def get_attempts_landed_columns(df):
    list_cols_attempts = []
    list_cols_landed = []
    for column in df.columns: 
        if '_attempts' in column.lower():
            list_cols_attempts.append(column)
        if '_landed' in column.lower():
            list_cols_landed.append(column)
    list_cols_attempts = [re.sub('_attempts','',col) for col in list_cols_attempts] # Remove "_Attempts"
    list_cols_landed = [re.sub('_landed','',col) for col in list_cols_landed]
    Attempts_Only = set(list_cols_attempts) - set(list_cols_landed)
    Landed_Only =  set(list_cols_landed) - set(list_cols_attempts)
    Both_Landed_Attempts = set(list_cols_landed) & set(list_cols_attempts)
    return Both_Landed_Attempts, Attempts_Only, Landed_Only

### Run the Function

In [9]:
df, Landed_Percent = Perform_calc_success_percent(df)

num columns before: 107
# Columns Added: 46
num columns after: 153


## Perform Transformation:  Calculate In Fight Differentials

In [10]:
f1_calc_columns = list(set(F1_Columns) - set(F1_Identification) - set(F1_TIP))

In [11]:
# I also need to get the percent columns
# At this point the df already has the percent columns... so just grab them from the df directly
percent_cols = [col[3:] for col in df.columns if 'percent' in col and 'f1' in col]

In [12]:
def Perform_calc_fight_stat_differential(df):
    percent_cols = [col[3:] for col in df.columns if 'percent' in col and 'f1' in col]
    f1_calc_columns = list(set(F1_Columns) - set(F1_Identification) - set(F1_TIP))
    for_calc_columns = [col[3:] for col in f1_calc_columns] # Remove the F1_ part
    for_calc_columns.extend(percent_cols)
    num_columns_before = df.shape[1]
    columns_before = df.columns
    print(f'num columns before: {num_columns_before}')
    for col in for_calc_columns:
        df = calc_fight_stat_differential(df, col)
    num_columns_after = df.shape[1]
    columns_after = df.columns
    print(f"# Columns Added: {num_columns_after-num_columns_before}")
    print(f'num columns after: {num_columns_after}')
    new_columns = list(set(columns_after) - set(columns_before))
    return df, new_columns

In [13]:
def calc_fight_stat_differential(df, col_root):
    f1 = 'f1_' + col_root
    f2 = 'f2_' + col_root
    diff = col_root + '_diff'
    df[diff] = df.apply(lambda x: (x[f1] - x[f2])/(x[f1] + x[f2]) if (x[f1] + x[f2]) != 0 else 0, axis=1)
    return df

# calc_fight_stat_differential(df, col_root)

### Run the Function

In [14]:
df, stat_diff = Perform_calc_fight_stat_differential(df)

num columns before: 153
# Columns Added: 71
num columns after: 224


## This dataset is currently missing the date.  Join it from the Events_df

In [15]:
path = '../../02_Data/01_Raw_Scraped_Data/Events/events_df.csv'

events_df = pd.read_csv(path, index_col=0)
events_df = events_df[['EventId','Date']]
events_df = events_df.rename(index=str, columns={"EventId": "eventid", 'Date':'date'})

In [16]:
df = df.merge(events_df, on='eventid')

In [17]:
df.shape

(4774, 225)

In [19]:
df.head()

,currentrnd,eventid,fightid,maxrnds,timestamp,f1_body_significant_strikes_attempts,f1_body_significant_strikes_landed,f1_body_total_strikes_attempts,f1_body_total_strikes_landed,f1_clinch_body_strikes_attempts,...,grappling_takedowns_percent_diff,total_strikes_percent_diff,distance_leg_strikes_percent_diff,distance_head_strikes_percent_diff,head_total_strikes_percent_diff,head_significant_strikes_percent_diff,clinch_total_strikes_percent_diff,body_significant_strikes_percent_diff,ground_leg_strikes_percent_diff,date
0,1,704,5268,3,02:26:26 06/28/2016,1.0,0.0,1.0,0.0,0.0,...,0.0,-0.179487,-0.111111,-0.440678,-0.455253,-0.455253,0.000000,-1.000000,0.0,2015-03-14
1,5,704,5198,5,02:49:04 03/16/2015,37.0,27.0,56.0,45.0,5.0,...,1.0,0.137835,-0.062500,0.034213,0.066588,0.181939,-0.067961,0.029797,1.0,2015-03-14
2,3,704,5199,3,02:49:03 03/16/2015,33.0,24.0,33.0,24.0,2.0,...,0.0,0.423800,0.463415,0.485306,0.402062,0.402062,0.052632,0.043478,0.0,2015-03-14
3,2,704,5249,3,02:49:08 03/16/2015,16.0,8.0,16.0,8.0,1.0,...,0.0,-0.233333,-0.032258,-0.301328,-0.345656,-0.345656,1.000000,0.047619,0.0,2015-03-14
4,3,704,5245,3,02:49:06 03/16/2015,9.0,7.0,12.0,10.0,5.0,...,-1.0,-0.074095,-1.000000,-0.243478,-0.095882,-0.084337,0.463415,-0.125000,0.0,2015-03-14


## Processing for V2 Df is done.  Export it

In [75]:
df.to_csv('../../02_Data/02_Processed_Data/V2_Processed.csv')

In [44]:
!ls ../../02_Data/02_Processed_Data/

V1_DF.csv                       df_ems.csv
V1_DF_w_flipped.csv             fighter_page_info.csv
V2_Fight_Details.csv            fighter_static_stats.csv
V2_Fight_Details_Munged0711.csv train.csv
V2_Processed.csv                train_stub.csv


In [67]:
# Training Stub Data
train_stub = ['eventid','fightid','f1_fullname','f2_fullname','f1_fighterid','f2_fighterid','f1_outcome']
train = V1_df[train_stub].reset_index().drop(columns='index')


df_ems_f1 = df_ems.copy()

# First append 'F1_' for all fighter 1 data
df_ems_f1.columns = ['eventid','fightid','f1_fighterid','date'] + \
                    ['f1_' + col for col in df_ems_f1.columns if col not in \
                    ['eventid','fightid','f1_fighterid','date']]

# Merge F1 Expanding Means
train = train.merge(df_ems_f1, left_on=['eventid','fightid','f1_fighterid'],
                    right_on=['eventid','fightid','f1_fighterid'])

#Setup 
df_ems_f2 = df_ems.drop(columns=['date']).copy()
df_ems_f2.columns = ['eventid','fightid','f2_fighterid'] + \
                    ['f2_' + col for col in df_ems_f2.columns if col not in \
                    ['eventid','fightid','f1_fighterid']]

# Merge em for fighter 2
train = train.merge(df_ems_f2, left_on=['eventid','fightid','f2_fighterid'], 
                    right_on=['eventid','fightid','f2_fighterid'])

# drop columns w/o outcome
train = train.dropna(axis=0, how='any')

# Check empty columns:
check_nulls(train)

Series([], dtype: float64)

# Need to clean up data a bit more before I can model
- Label from win/loss to 1/0

In [69]:
train.f1_outcome.value_counts()

Win     1590
Loss    1590
Name: f1_outcome, dtype: int64

In [71]:
train['outcome'] = train.f1_outcome.map(lambda x: 1 if x=='Win' else 0)

# Export Train dataset for some EDA

In [72]:
train.to_csv('../../02_Data/02_Processed_Data/train.csv')